In [1]:
import datetime
import math
import os
import sys
import ee
from Py6S import *
import ee
import geemap

import geemap
import ee
from IPython.display import display

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfJohXk50mInsZNnH1maZu0XWECM-LGDBsZQB1kyANokkKI6oGwr09iUsDE

Successfully saved authorization token.


In [3]:
import os
import geemap
import ee

In [4]:
import os
import geemap
import ee

def export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_dir = "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/"
                os.makedirs(landsat_output_dir, exist_ok=True)
                landsat_output_path = os.path.join(landsat_output_dir, landsat_filename)
                geemap.ee_export_image(image, landsat_output_path, region=jharkhand, scale=30, file_per_band=False)

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDVI/"
                evi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/EVI/"
                ndwi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDWI/"

                os.makedirs(ndvi_output_dir, exist_ok=True)
                os.makedirs(evi_output_dir, exist_ok=True)
                os.makedirs(ndwi_output_dir, exist_ok=True)

                # Export NDVI, EVI, and NDWI to separate folders
                ndvi_output_path = os.path.join(ndvi_output_dir, ndvi_filename)
                evi_output_path = os.path.join(evi_output_dir, evi_filename)
                ndwi_output_path = os.path.join(ndwi_output_dir, ndwi_filename)

                geemap.ee_export_image(ndvi, ndvi_output_path, region=jharkhand, scale=30)
                geemap.ee_export_image(evi, evi_output_path, region=jharkhand, scale=30)
                geemap.ee_export_image(ndwi, ndwi_output_path, region=jharkhand, scale=30)

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1984
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_dir = '/path/to/output/directory'
cloudCover = 30

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover)


No images found for year 1984. Moving to the next year.
No images found for year 1985. Moving to the next year.
No images found for year 1986. Moving to the next year.
No images found for year 1987. Moving to the next year.
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1988_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1988_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\

Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1997_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1997_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1998_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data d

Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2006_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2006_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (970248510 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2007_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2007_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total re

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2015_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2015_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (1068253410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2016_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2016_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (106

In [7]:

import os
import geemap
import ee

def export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                #images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Check if the image extent covers the full extent of the region
                if image.geometry().contains(jharkhand):
                    # Define output filename for Landsat
                    landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                    landsat_output_dir = "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/"
                    os.makedirs(landsat_output_dir, exist_ok=True)
                    landsat_output_path = os.path.join(landsat_output_dir, landsat_filename)
                    geemap.ee_export_image(image, landsat_output_path, region=jharkhand, scale=30, file_per_band=False)

                    # Calculate NDVI, EVI, and NDWI
                    band_info = band_mapping[landsat_sensor]
                    ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                    evi = image.expression(
                        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                            'NIR': image.select(band_info['NIR']),
                            'RED': image.select(band_info['RED']),
                            'BLUE': image.select(band_info['BLUE'])
                        })
                    ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                    # Define output filenames for NDVI, EVI, and NDWI
                    ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                    evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                    ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                    ndvi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDVI/"
                    evi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/EVI/"
                    ndwi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDWI/"

                    os.makedirs(ndvi_output_dir, exist_ok=True)
                    os.makedirs(evi_output_dir, exist_ok=True)
                    os.makedirs(ndwi_output_dir, exist_ok=True)

                    # Export NDVI, EVI, and NDWI to separate folders
                    ndvi_output_path = os.path.join(ndvi_output_dir, ndvi_filename)
                    evi_output_path = os.path.join(evi_output_dir, evi_filename)
                    ndwi_output_path = os.path.join(ndwi_output_dir, ndwi_filename)

                    geemap.ee_export_image(ndvi, ndvi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(evi, evi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(ndwi, ndwi_output_path, region=jharkhand, scale=30)

                    break  # Exit the loop when an image is found for the current year and month

        if not images_found:
            # If no images are found for the current year, continue to the next year
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1984
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_dir = '/path/to/output/directory'
cloudCover = 30

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until a suitable image is found
export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover)


No images found for year 1984. Moving to the next year.
No images found for year 1985. Moving to the next year.
No images found for year 1986. Moving to the next year.
No images found for year 1987. Moving to the next year.
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1988_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1988_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1996_02_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1997_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1997_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1998_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (882

Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2006_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2006_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (970248510 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2007_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2007_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total re

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2015_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2015_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (1068253410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2016_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2016_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (106

In [10]:
import os
import geemap
import ee

def export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Start with an empty image collection for the current year
        images = ee.ImageCollection([])

        # Iterate through months from January to December
        for month in range(1, 13):
            # Calculate the month to use, considering wrapping from December to January
            current_month = month if month <= 12 else month - 12
            current_year = year if month <= 12 else year + 1

            # Set the start and end dates for the current month
            start_date = f'{current_year}-{current_month:02d}-01'
            end_date = f'{current_year}-{current_month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = images.merge(
                landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover))
                .filter(ee.Filter.date(start_date, end_date))
                .filterBounds(jharkhand)
            )

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                # Take the mean of the images for the specified period
                image = images.mean()

                # Check if the image extent covers the full extent of the region
                if image.geometry().contains(jharkhand):
                    images_found = True

                    # Define output filename for Landsat
                    landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                    landsat_output_dir = "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/"
                    os.makedirs(landsat_output_dir, exist_ok=True)
                    landsat_output_path = os.path.join(landsat_output_dir, landsat_filename)
                    geemap.ee_export_image(image, landsat_output_path, region=jharkhand, scale=30, file_per_band=False)

                    # Calculate NDVI, EVI, and NDWI
                    band_info = band_mapping[landsat_sensor]
                    ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                    evi = image.expression(
                        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                            'NIR': image.select(band_info['NIR']),
                            'RED': image.select(band_info['RED']),
                            'BLUE': image.select(band_info['BLUE'])
                        })
                    ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                    # Define output filenames for NDVI, EVI, and NDWI
                    ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                    evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                    ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                    ndvi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDVI/"
                    evi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/EVI/"
                    ndwi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDWI/"

                    os.makedirs(ndvi_output_dir, exist_ok=True)
                    os.makedirs(evi_output_dir, exist_ok=True)
                    os.makedirs(ndwi_output_dir, exist_ok=True)

                    # Export NDVI, EVI, and NDWI to separate folders
                    ndvi_output_path = os.path.join(ndvi_output_dir, ndvi_filename)
                    evi_output_path = os.path.join(evi_output_dir, evi_filename)
                    ndwi_output_path = os.path.join(ndwi_output_dir, ndwi_filename)

                    geemap.ee_export_image(ndvi, ndvi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(evi, evi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(ndwi, ndwi_output_path, region=jharkhand, scale=30)

                break  # Exit the loop when an image is found for the current year and month

        if not images_found:
            # If no images are found for the current year, continue to the next year
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1990
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_dir = '/path/to/output/directory'
cloudCover = 30

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until a suitable image is found
export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover)


Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1990_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1990_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1991_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data d

Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1999_10_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1999_10_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (970248510 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2000_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2000_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total re

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2008_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2008_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (970248510 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2009_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2009_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (9702

An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2017_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (1068253410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2018_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2018_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (1068253410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\J

In [11]:
import os
import geemap
import ee

def export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Start with an empty image collection for the current year
        images = ee.ImageCollection([])

        # Iterate through months from January to December
        for month in range(1, 13):
            # Calculate the month to use, considering wrapping from December to January
            current_month = month if month <= 12 else month - 12
            current_year = year if month <= 12 else year + 1

            # Set the start and end dates for the current month
            start_date = f'{current_year}-{current_month:02d}-01'
            end_date = f'{current_year}-{current_month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = images.merge(
                landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover))
                .filter(ee.Filter.date(start_date, end_date))
                .filterBounds(jharkhand)
            )

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                # Take the mean of the images for the specified period
                image = images.mean()

                # Check if the image extent covers the full extent of the region
                if image.geometry().contains(jharkhand):
                    images_found = True

                    # Define output filename for Landsat
                    landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                    landsat_output_dir = "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/"
                    os.makedirs(landsat_output_dir, exist_ok=True)
                    landsat_output_path = os.path.join(landsat_output_dir, landsat_filename)
                    geemap.ee_export_image(image, landsat_output_path, region=jharkhand, scale=30, file_per_band=False)

                    # Calculate NDVI, EVI, and NDWI
                    band_info = band_mapping[landsat_sensor]
                    ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                    evi = image.expression(
                        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                            'NIR': image.select(band_info['NIR']),
                            'RED': image.select(band_info['RED']),
                            'BLUE': image.select(band_info['BLUE'])
                        })
                    ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                    # Define output filenames for NDVI, EVI, and NDWI
                    ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                    evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                    ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                    ndvi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDVI/"
                    evi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/EVI/"
                    ndwi_output_dir =  "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDWI/"

                    os.makedirs(ndvi_output_dir, exist_ok=True)
                    os.makedirs(evi_output_dir, exist_ok=True)
                    os.makedirs(ndwi_output_dir, exist_ok=True)

                    # Export NDVI, EVI, and NDWI to separate folders
                    ndvi_output_path = os.path.join(ndvi_output_dir, ndvi_filename)
                    evi_output_path = os.path.join(evi_output_dir, evi_filename)
                    ndwi_output_path = os.path.join(ndwi_output_dir, ndwi_filename)

                    geemap.ee_export_image(ndvi, ndvi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(evi, evi_output_path, region=jharkhand, scale=30)
                    geemap.ee_export_image(ndwi, ndwi_output_path, region=jharkhand, scale=30)

                    break  # Exit the loop when an image is found for the current year and month

        if not images_found:
            # If no images are found for the current year, continue to the next year
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1984
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_dir = '/path/to/output/directory'
cloudCover = 30

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until a suitable image is found
export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover)


No images found for year 1984. Moving to the next year.
No images found for year 1985. Moving to the next year.
No images found for year 1986. Moving to the next year.
No images found for year 1987. Moving to the next year.
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1988_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1988_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1996_02_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1997_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_1997_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (872243610 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_1998_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (882

Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2006_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2006_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (970248510 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2007_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2007_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total re

Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2015_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2015_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (1068253410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDVI\Landsat_2016_01_NDVI.tif
Generating URL ...
An error occurred while downloading.
Total request size (88204410 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to D:\Jintu\Jintu_ShareFolder\TestDL_2\Landsat Images\NDWI\Landsat_2016_01_NDWI.tif
Generating URL ...
An error occurred while downloading.
Total request size (106

In [14]:
import ee
import os
import time

# Initialize Earth Engine
ee.Initialize()

# Define the region of interest (ROI) as a geometry
table = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()

# Define the NIR band name for different Landsat sensors
def get_nir_band(landsat_sensor):
    if landsat_sensor in ['LT05', 'LE07']:
        return 'B4'
    elif landsat_sensor in ['LC08', 'LC09']:
        return 'B5'
    else:
        raise ValueError(f"Unsupported Landsat sensor: {landsat_sensor}")

# Loop through the years from 1984 to 2023
for year in range(2000, 2001):
    images_found = False  # Flag to check if images are found for the current year

    # Get the Landsat sensor based on the year
    landsat_sensor = None
    if year < 1999:
        landsat_sensor = 'LT05'
    elif 1999 <= year < 2014:
        landsat_sensor = 'LE07'
    elif 2014 <= year < 2022:
        landsat_sensor = 'LC08'
    else:
        landsat_sensor = 'LC09'

    # Initialize GLCM variables for each texture property
    glcm_mean_contrast = None
    glcm_mean_correlation = None
    glcm_mean_dissimilarity = None
    glcm_mean_energy = None
    glcm_mean_entropy = None
    glcm_count = 0

    # Initialize a flag to check if any NaN values are found
    nan_found = False

    # Iterate through months from January to December
    for month in range(1, 13):
        # Check if NaN values are found in the images
        if nan_found:
            print(f"NaN value found in year {year}, month {month}. Adding the next month and taking the mean.")
        else:
            # Set the start and end dates for the current month
            start_date = f'{year}-01-01'
            end_date = f'{year}-12-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            
            images_1 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(table)\
                     .filter(ee.Filter.eq('WRS_ROW',41))\
                     .filter(ee.Filter.eq('WRS_ROW',41))\
                     .filter(ee.Filter.eq('WRS_PATH',136))\
                     .first()
            # Get the date of the first image in images_1
            first_image_date = ee.Image(images_1).date()
            

            images_2 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(first_image_date, first_image_date.advance(1, 'day'))) \
                     .filterBounds(table)\
                     .filter(ee.Filter.eq('WRS_ROW',42))\
                     .filter(ee.Filter.eq('WRS_ROW',42))\
                     .filter(ee.Filter.eq('WRS_PATH',136))\
                     .first()
                
            #images = images_1.merge(images_2)
            images = ee.ImageCollection([images_1, images_2])
                
            

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mosaic()


                # Select NIR band for GLCM
                nir_band = get_nir_band(landsat_sensor)
                nir_image = image.select(nir_band)

                # Convert NIR band to 32-bit integer
                nir_image_int = nir_image.multiply(10000).toInt32()

                # Calculate GLCM texture properties
                texture = nir_image_int.glcmTexture()
                
                print(texture.getInfo())

                # Select GLCM bands
                contrast = texture.select(get_nir_band(landsat_sensor)+'_contrast')
                correlation = texture.select(get_nir_band(landsat_sensor)+'_corr')
                dissimilarity = texture.select(get_nir_band(landsat_sensor)+'_diss')
                energy = texture.select(get_nir_band(landsat_sensor)+'_energy')
                entropy = texture.select(get_nir_band(landsat_sensor)+'_entropy')

                # Update GLCM means for each texture property
                if glcm_mean_contrast is None:
                    glcm_mean_contrast = contrast
                    glcm_mean_correlation = correlation
                    glcm_mean_dissimilarity = dissimilarity
                    glcm_mean_energy = energy
                    glcm_mean_entropy = entropy
                else:
                    glcm_mean_contrast = glcm_mean_contrast.add(contrast)
                    glcm_mean_correlation = glcm_mean_correlation.add(correlation)
                    glcm_mean_dissimilarity = glcm_mean_dissimilarity.add(dissimilarity)
                    glcm_mean_energy = glcm_mean_energy.add(energy)
                    glcm_mean_entropy = glcm_mean_entropy.add(entropy)
                
                glcm_count += 1

                # Check for NaN values in the contrast image
                if contrast.reduceRegion(reducer=ee.Reducer.anyNonZero(), geometry=table, scale=30).getInfo():
                    nan_found = True
                    print(f"NaN value found in year {year}, month {month}. Adding the next month and taking the mean.")
                    break  # Exit the loop for the current year and month

    # Calculate the mean of GLCM texture properties across all valid months
    if glcm_mean_contrast is not None and glcm_count > 0:
        glcm_mean_contrast = glcm_mean_contrast.divide(glcm_count)
        glcm_mean_correlation = glcm_mean_correlation.divide(glcm_count)
        glcm_mean_dissimilarity = glcm_mean_dissimilarity.divide(glcm_count)
        glcm_mean_energy = glcm_mean_energy.divide(glcm_count)
        glcm_mean_entropy = glcm_mean_entropy.divide(glcm_count)

        # Clip the GLCM images to the ROI
        glcm_mean_contrast = glcm_mean_contrast.clip(table)
        glcm_mean_correlation = glcm_mean_correlation.clip(table)
        glcm_mean_dissimilarity = glcm_mean_dissimilarity.clip(table)
        glcm_mean_energy = glcm_mean_energy.clip(table)
        glcm_mean_entropy = glcm_mean_entropy.clip(table)

        # Define the output folder
        output_folder = 'GEEexport'

        # Define output filenames for GLCM texture properties
        glcm_contrast_filename = f'GLCM_contrast_{year}.tif'
        glcm_correlation_filename = f'GLCM_correlation_{year}.tif'
        glcm_dissimilarity_filename = f'GLCM_dissimilarity_{year}.tif'
        glcm_energy_filename = f'GLCM_energy_{year}.tif'
        glcm_entropy_filename = f'GLCM_entropy_{year}.tif'

        # Export options
        export_options = {
            'scale': 30,
            'region': table,
            'crs': 'EPSG:32644',  # Change to your desired CRS
            'maxPixels': 1e13  # Change to an appropriate value
        }

        # Export GLCM texture properties to Google Drive with export options
        export_glcm_contrast_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_contrast,
            description='GLCM Contrast Export',
            folder=output_folder,
            fileNamePrefix=glcm_contrast_filename,
            **export_options  # Include export options
        )

        export_glcm_correlation_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_correlation,
            description='GLCM Correlation Export',
            folder=output_folder,
            fileNamePrefix=glcm_correlation_filename,
            **export_options  # Include export options
        )

        export_glcm_dissimilarity_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_dissimilarity,
            description='GLCM Dissimilarity Export',
            folder=output_folder,
            fileNamePrefix=glcm_dissimilarity_filename,
            **export_options  # Include export options
        )

        export_glcm_energy_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_energy,
            description='GLCM Energy Export',
            folder=output_folder,
            fileNamePrefix=glcm_energy_filename,
            **export_options  # Include export options
        )

        export_glcm_entropy_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_entropy,
            description='GLCM Entropy Export',
            folder=output_folder,
            fileNamePrefix=glcm_entropy_filename,
            **export_options  # Include export options
        )

        # Start export tasks
        export_glcm_contrast_task.start()
        export_glcm_correlation_task.start()
        export_glcm_dissimilarity_task.start()
        export_glcm_energy_task.start()
        export_glcm_entropy_task.start()

        print(f'Exporting GLCM texture properties for {year} to Google Drive...')

        # Wait for export tasks to complete
        while (export_glcm_contrast_task.active() or export_glcm_correlation_task.active()
               or export_glcm_dissimilarity_task.active() or export_glcm_energy_task.active()
               or export_glcm_entropy_task.active()):
            print('Exporting...')  # You can replace this with a more informative message
            time.sleep(30)  # Wait for 30 seconds before checking again

        print(f'Export completed for GLCM texture properties for {year}')
    elif not images_found:
        print(f"No images found for year {year}. Moving to the next year.")
        
        


{'type': 'Image', 'bands': [{'id': 'B4_asm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_contrast', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_corr', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_var', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_idm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_savg', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_svar', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_sent', 'data_ty

In [7]:
# Example usage:
start_year = 2005
end_year = 2015
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 20
pathNumber = 136
rowNumber1 = 41
rowNumber2=42


import datetime
import math
import os
import sys
import ee
from Py6S import *
import ee
import geemap
import ee
import folium
from IPython.display import display
import numpy as np
import geemap
import ee
from IPython.display import display


import ee
import numpy as np
import matplotlib.pyplot as plt
from geemap import ee_export_image
import time
import ee
import geemap
import os